参考：https://github.com/xecus/tagurobot-v1

![](https://cdn-ak.f.st-hatena.com/images/fotolife/h/hiroyuki_abeja/20231204/20231204211830.png)

![](https://drive.google.com/uc?id=1uv3vFMY5C87Ol3BcQ3uRj85PeiE7iSK9)


In [6]:
import enum
import math
import sys
import time
import numpy as np
import pandas as pd

class JointGroupId(enum.IntEnum):
    A = 0
    B = 1
    C = 2
    D = 3
    E = 4
    F = 5

class JointType(enum.IntEnum):
    J1 = 0
    J2 = 1
    J3 = 2
    J4 = 3

# よく使う計算ロジック（回転行列の計算）
class CalcUtil():
    @classmethod
    def rotation(cls, u, t):
        r = np.array([[np.cos(t), -np.sin(t)], [np.sin(t), np.cos(t)]])
        return np.dot(r, u)

class Hexapod():
    def __init__(self):
        self.init_position()

    def init_position(self):
        # Hexapodの諸元
        self.side_length = 0.1
        self.long1 = [0.04 for i in range(6)]
        self.long2 = [0.08 for i in range(6)]
        self.long3 = [0.08 for i in range(6)]
        # 各サーボモーターの角度 (シミュレーション上)
        self.theta1 = [math.radians(0) for i in range(6)]
        self.theta2 = [math.radians(0) for i in range(6)]
        self.theta3 = [math.radians(0) for i in range(6)]
        # X-Y座標面 (TOP VIEWからの座標面)
        self.leg_coords_topview = [[(0, 0) for j in range(6)] for i in range(4)]
        # X-Z'座標面 (各リンク機構をSIDE VIEWしたもの)
        self.leg_coords_sideview = [[(0, 0) for j in range(6)] for i in range(4)]
        # 順方向計算に使うパラメータ
        self.real_coords = [(0.15, -0.08) for i in range(6)]
        self.distance = [0.15 for i in range(6)]
        self.height = [-0.08 for i in range(6)]
        # 初期位置の決定
        for group_id in range(6):
            self.calc_backward(group_id)

    # 各関節角度のExport (サーボ制御系に渡すため)
    def export_angles(self):
        return [self.theta1, self.theta2, self.theta3]

    # 逆方向計算
    # 与えられた座標に持っていくために必要な関節角度(Theta2, Theta3)の計算
    # ※Theta1は別の場所で計算
    def calc_backward(self, group_id, apply=True):
        distance = self.distance[group_id]
        height = self.height[group_id]
        (real_x, real_y) = self.real_coords[group_id]
        real_x = distance if distance is not None else real_x
        real_y = height if height is not None else real_y

        l2 = self.long2[group_id]
        l3 = self.long3[group_id]
        real_x = real_x - self.long1[group_id]
        L = math.sqrt(real_x * real_x + real_y * real_y)
        J3, B, A, J2 = None, None, None, None
        ret = None
        try:
            J3 = math.acos((l2 * l2 + l3 * l3 - L * L) / (2 * l2 * l3))
            B = math.acos((L * L + l2 * l2 - l3 * l3) / (2 * L * l2))
            A = math.fabs(math.atan(real_y / real_x))
            # A = math.atan2(real_y, real_x)
            J2 = B - A
            theta2 = J2
            theta3 = -(math.pi - J3)
            ret = (theta2, theta3)
            if apply:
                self.theta2[group_id] = theta2
                self.theta3[group_id] = theta3

        except ValueError:
            return ret
        except ZeroDivisionError:
            return ret

        # for debug
        # print(group_id,B,A,J2,J3,theta2,theta3, math.degrees(theta2), math.degrees(theta3))

        return ret

    # 順方向計算
    # 与えられた構造的パラメータと角度(Theta1, Theta2等)から関節位置を算出
    def calc_forward(self):
        r = self.side_length
        step = (math.pi * 2.0) / 6

        for i in range(6):
            l1 = self.long1[i]
            l2 = self.long2[i]
            l3 = self.long3[i]

            # TOP View
            x = r * math.cos(step * i)
            y = r * math.sin(step * i)
            self.leg_coords_topview[JointType.J1][i] = (x, y)

            Ru1 = CalcUtil.rotation((l1, 0), step * i - self.theta1[i])
            x = r * math.cos(step * i) + Ru1[0]
            y = r * math.sin(step * i) + Ru1[1]
            self.leg_coords_topview[JointType.J2][i] = (x, y)

            l2_top = CalcUtil.rotation((l2, 0), self.theta2[i])
            Ru2 = CalcUtil.rotation((l2_top[0], 0), step * i - self.theta1[i])
            x = r * math.cos(step * i) + Ru1[0] + Ru2[0]
            y = r * math.sin(step * i) + Ru1[1] + Ru2[1]
            self.leg_coords_topview[JointType.J3][i] = (x, y)

            l3_top = CalcUtil.rotation(
                (l3, 0), self.theta2[i] + self.theta3[i])
            Ru3 = CalcUtil.rotation((l3_top[0], 0), step * i - self.theta1[i])
            x = r * math.cos(step * i) + Ru1[0] + Ru2[0] + Ru3[0]
            y = r * math.sin(step * i) + Ru1[1] + Ru2[1] + Ru3[1]
            self.leg_coords_topview[JointType.J4][i] = (x, y)

            # Side View
            x = 0
            y = 0
            self.leg_coords_sideview[JointType.J1][i] = (x, y)

            x = l1
            y = 0
            self.leg_coords_sideview[JointType.J2][i] = (x, y)

            Ru1 = CalcUtil.rotation((l2, 0), self.theta2[i])
            x = self.long1[i] + Ru1[0]
            y = -Ru1[1]
            self.leg_coords_sideview[JointType.J3][i] = (x, y)

            Ru1 = CalcUtil.rotation((self.long2[i], 0), self.theta2[i])
            Ru2 = CalcUtil.rotation(
                (self.long3[i], 0), self.theta2[i] + self.theta3[i])
            x = self.long1[i] + Ru1[0] + Ru2[0]
            y = - (Ru1[1] + Ru2[1])
            self.leg_coords_sideview[JointType.J4][i] = (x, y)

class mainApp():
    def __init__(self):
        self.cnt = 0
        self.hexapod = Hexapod()
        # self.df = pd.Series([])
        self.df = pd.DataFrame(columns=['Count', 'Joint', 'Theta1', 'Theta2', 'Theta3','Distance', 'Height'])

    def calc(self, count):
        self.cnt = count
        # self.hexapod.calc_forward()

        def relu(a):
            return a if a > 0 else 0

        # 1周期の分割数 (高ければ高いほど小刻みな運動)
        num_divide = 10 # 100
        # 一歩の歩幅 (m)
        stroke_step_length_a = 0.03
        # 一歩の高さ
        stroke_step_length_b = 0.01
        # TOP_VIEWにおける関節と足の定位置までの距離
        distance_a = 0.1
        # SIDE_VIEWから見た時の関節と地面までの距離
        distance_b = -0.08
        # よく使い回す値
        tmp1 = 2.0 * math.pi / 6.0
        tmp2 = 2.0 * math.pi / num_divide
        # 各関節の位置計算
        for target_joint in range(6):
            # 位相角の計算
            phase_diff1 = math.pi / 2 if (target_joint % 2 != 0) else 0
            phase_diff2 = math.pi if (target_joint % 2 != 0) else 0
            # J1からdistance_a[m]離れた位置を計算する (TOP_VIEWでみた足を着地すべき場所)
            (x1, y1) = self.hexapod.leg_coords_topview[JointType.J1][target_joint]
            x = distance_a * math.cos(tmp1 * target_joint)
            y = distance_a * math.sin(tmp1 * target_joint)
            y -= stroke_step_length_a * math.fabs(math.sin(tmp2 * self.cnt + phase_diff1))
            # 角度、距離などを変更
            self.hexapod.theta1[target_joint] = math.atan2(-y, x) + (2.0 * math.pi / 6.0) * target_joint
            self.hexapod.distance[target_joint] = math.sqrt(x * x + y * y)
            self.hexapod.height[target_joint] = distance_b + math.fabs(relu(stroke_step_length_b * math.sin(tmp2 * 2 * self.cnt + phase_diff2)))

        # 各関節角度の計算 (逆方向計算)
        for i in range(6):
            self.hexapod.calc_backward(i)
        # 各関節位置の計算 (順方向計算)
        self.hexapod.calc_forward()

        # 座標系補正(サーボモータ中点90度を中心に振る)
        servo_center_leg_deg = 90.0 # 90.0
        servo_center_foot_deg = 60.0 # 90.0
        for i in range(6):
            self.hexapod.theta1[i] = (math.radians(servo_center_leg_deg) - self.hexapod.theta1[i]) if (i<3) else (math.radians(servo_center_leg_deg) - (self.hexapod.theta1[i] - math.radians(270)) + math.radians(servo_center_leg_deg))
            self.hexapod.theta2[i] = (math.radians(servo_center_foot_deg) - self.hexapod.theta2[i]) if (i<3) else (math.radians(servo_center_foot_deg) - self.hexapod.theta2[i])

        # 内部角度情報のエクスポート
        angles = self.hexapod.export_angles()

        # データ出力
        for i_joint in range(6):

            row_df = pd.Series([
                self.cnt,
                i_joint,
                math.degrees(self.hexapod.theta1[i_joint]),
                math.degrees(self.hexapod.theta2[i_joint]),
                math.degrees(self.hexapod.theta3[i_joint]),
                self.hexapod.distance[i_joint],
                self.hexapod.height[i_joint],
            ], index=['Count', 'Joint', 'Theta1', 'Theta2', 'Theta3', 'Distance', 'Height'])
            self.df = pd.concat([self.df, row_df.to_frame().T])

        self.df_sorted = self.df.set_index(['Count', self.df.groupby('Count').cumcount()]).unstack().sort_index(level=1, axis=1)
        self.df_sorted.columns = [f'{col}{joint + 1}' for col, joint in self.df_sorted.columns]

    def print_data(self):
        print(self.df)

    def print_data_sorted(self):
        print(self.df_sorted)

    def print_df_sorted(self):
        print(self.df_sorted)
    def to_csv(self, file_path):
        self.df.to_csv(file_path, float_format='%.3f')

    def to_csv_sorted(self, file_path):
        self.df_sorted.to_csv(file_path, float_format='%.3f')

    # C++構造体のコードを生成
    def generate_cpp_struct(self, file_path):
        df = pd.read_csv(file_path)

        print('\n\n')
        print('float pattern_walk[][1+NUM_LEG_STRUCTURE*NUM_LEGS] = {')
        # print('float pattern_walk[][1+3*NUM_LEGS] = {')   # theta3は使わない
        for index, row in df.iterrows():
            count = [int(row.Count) for row in df.itertuples(index=False)]
            # distances = [row[f'Distance{i}'] for i in range(1, 7)]
            # heights = [row[f'Height{i}'] for i in range(1, 7)]
            # joints = [int(row[f'Joint{i}']+1) for i in range(1, 7)]
            thetas = [(row[f'Theta{j}{i}']) for i in range(1, 7) for j in range(1, 3)]  # theta3は使わない
            # thetas = [(row[f'Theta{j}{i}']) for i in range(1, 7) for j in range(1, 4)]
            print('    { ', end='')
            print(f'{int(row.Count)} ', *thetas, sep=',', end='')
            print(' },')
        print('};')


In [9]:
if __name__ == '__main__':
    app = mainApp()

    frames = 10 # 50
    for count in range(0, frames):
        app.calc(count)

    # app.print_data()
    app.print_df_sorted()

    from datetime import datetime
    import pytz
    jst = pytz.timezone('Asia/Tokyo')
    current_datetime = datetime.now(jst)

    file_name = current_datetime.strftime('hexpod_tripod_gait' + '%Y%m%d-%H%M%S.csv')
    file_path = '' + file_name

    # app.to_csv(file_path)
    app.to_csv_sorted(file_path)

    app.generate_cpp_struct(file_path)


       Distance1   Height1  Joint1    Theta11    Theta21     Theta31  \
Count                                                                  
0.0     0.100000 -0.080000     0.0  90.000000  61.812290 -102.635625   
1.0     0.101543 -0.070489     0.0  79.999522  54.668434 -108.416108   
2.0     0.103991 -0.074122     0.0  74.075654  56.931086 -104.528845   
3.0     0.103991 -0.080000     0.0  74.075654  61.156671 -100.375199   
4.0     0.101543 -0.080000     0.0  79.999522  61.541036 -101.776830   
5.0     0.100000 -0.080000     0.0  90.000000  61.812290 -102.635625   
6.0     0.101543 -0.070489     0.0  79.999522  54.668434 -108.416108   
7.0     0.103991 -0.074122     0.0  74.075654  56.931086 -104.528845   
8.0     0.103991 -0.080000     0.0  74.075654  61.156671 -100.375199   
9.0     0.101543 -0.080000     0.0  79.999522  61.541036 -101.776830   

       Distance2   Height2  Joint2    Theta12  ...  Joint5    Theta15  \
Count                                          ...            

----
テストコード置き場

In [ ]:
df = pd.read_csv('./hexpod_tripod_gait20240414-195744.csv')

print('tripod_gait_pattern = {')
for index, row in df.iterrows():
    count = [int(row.Count) for row in df.itertuples(index=False)]
    # distances = [row[f'Distance{i}'] for i in range(1, 7)]
    # heights = [row[f'Height{i}'] for i in range(1, 7)]
    # joints = [int(row[f'Joint{i}']+1) for i in range(1, 7)]
    thetas = [(row[f'Theta{j}{i}']) for i in range(1, 7) for j in range(1, 4)]
    print('    { ', end='')
    print(f'{int(row.Count)} ', *thetas, sep=',', end='')
    print(' },')
print('};')

tripod_gait_pattern = {
    { 1 ,1.079,-0.917,-103.54,11.426,-9.204,-113.651,-0.549,-1.232,-104.438,343.332,-1.1,-100.144,359.469,-0.632,-102.636,366.779,-0.208,-84.904 },
    { 2 ,2.153,-0.025,-104.4,11.337,-9.148,-113.605,-1.113,-0.688,-106.194,343.425,-1.107,-100.174,358.957,0.515,-102.582,366.748,-0.199,-85.031 },
    { 3 ,3.217,0.848,-105.202,11.19,-9.055,-113.527,-1.692,-0.192,-107.887,343.58,-1.119,-100.223,358.465,1.612,-102.464,366.695,-0.183,-85.241 },
    { 4 ,4.267,1.689,-105.935,10.987,-8.925,-113.418,-2.284,0.244,-109.502,343.797,-1.136,-100.29,357.993,2.643,-102.277,366.621,-0.162,-85.533 },
    { 5 ,5.296,2.483,-106.588,10.729,-8.76,-113.277,-2.885,0.611,-111.027,344.076,-1.157,-100.375,357.543,3.594,-102.015,366.526,-0.137,-85.905 },
    { 6 ,6.302,3.218,-107.153,10.42,-8.56,-113.103,-3.494,0.897,-112.448,344.414,-1.182,-100.476,357.115,4.453,-101.673,366.409,-0.109,-86.352 },
    { 7 ,7.279,3.881,-107.623,10.062,-8.328,-112.895,-4.107,1.096,-113.754,344.813,-1.211,-10